In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from tqdm.notebook import tqdm

nickname = '다녀5세요'
url = f'https://www.op.gg/summoners/kr/{nickname}'

# Chrome 드라이버 경로 설정
driver = webdriver.Chrome()

# op.gg 웹 사이트로 이동
print('로딩중...')
driver.get(url)

look = driver.find_element(By.CSS_SELECTOR, ".css-n9kjrp .header-profile-info .info > .last-update").text
print(look)

if re.search(r'(\d+)+(시간|일)\s+전', look):
    driver.find_element(By.CSS_SELECTOR, ".css-1ki6o6m.e18vylim0").click()
    print('데이터 최신화 중...')
    new = tqdm(range(5),desc='5초 소요',leave=False)
    for i in new:
        time.sleep(1)
    new.close()
        

driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(0.5)

for i in range(1,5):

    if driver.find_element(By.CSS_SELECTOR, '.css-1s9fubg .more'):  # 더 보기 버튼 찾기
        driver.find_element(By.CSS_SELECTOR, '.css-1s9fubg .more').click()  # 누르기
        print(f'게임 로드중...{i*20+20}/100')
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
    else:
        break
# details = driver.find_elements(By.XPATH, ("//div[contains(@class,'StatsButton')]"))
# for i in details:
#     i.click()
#     time.sleep(1)


    
print('완료')

html = driver.page_source
print('파일저장중')
f = open('/Users/dryoon/Documents/schoolproject/all.html','w')
f.write(html)
f.close() 
print('저장완료!')
driver.quit()


로딩중...
최근 업데이트: 5시간 전
데이터 최신화 중...


5초 소요:   0%|          | 0/5 [00:00<?, ?it/s]

게임 로드중...40/100
게임 로드중...60/100
게임 로드중...80/100
게임 로드중...100/100
완료
파일저장중
저장완료!


In [4]:
from bs4 import BeautifulSoup

# 파일 열기 (읽기 모드)
file_path = "/Users/dryoon/Documents/schoolproject/all.html"  # 파일 경로를 지정합니다.
file = open(file_path, "r")


# BeautifulSoup을 사용하여 HTML 파싱
soup = BeautifulSoup(file, 'html.parser')

# "game-content" 클래스 선택
game_content = soup.select('div.game-content')

recent_game_len = len(game_content)
print(f'가져온 게임의 판수는 {recent_game_len}판 입니다')

result_all = soup.find_all(class_="result")

# 추출한 결과를 리스트로 저장
result_values = [result.get_text() for result in result_all]

# 데이터프레임으로 변환
df = pd.DataFrame({'결과': result_values})

# 데이터프레임 출력
print(df)


file.close()

가져온 게임의 판수는 100판 입니다
    결과
0   승리
1   패배
2   승리
3   패배
4   승리
..  ..
95  승리
96  승리
97  승리
98  승리
99  승리

[100 rows x 1 columns]
